In [1]:
# FIRST PAGE: https://newsinfo.inquirer.net/category/inquirer-headlines/nation/1
# lAST PAGE: https://newsinfo.inquirer.net/category/inquirer-headlines/nation/page/4579

# Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import nltk
from nltk import tokenize
nltk.download('punkt')
import json
import time
import datetime
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mykog\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
headers = {"Content-Type": "application/x-www-form-urlencoded; charset=utf-8",
       "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"}


In [4]:
%%time

def getLinks(pageCount):
    page = requests.post("https://newsinfo.inquirer.net/category/nation/page/"+str(pageCount), headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    
    if(soup.find(id="channel-lbl") is not None):
        links = [i.a['href'] for i in soup.find_all("h2") if i.a is not None]
        return [{"link":j} for j in links][:9]
    else:
        return []
    
getLinks(121)

Wall time: 1.43 s


[{'link': 'https://newsinfo.inquirer.net/1669864/bill-seeks-to-criminalize-abuses-vs-elderly'},
 {'link': 'https://newsinfo.inquirer.net/1669547/dti-wants-supply-of-p70-sugar-to-last-till-post-christmas'},
 {'link': 'https://newsinfo.inquirer.net/1669556/makabayan-bloc-seeks-p40-b-budget-for-poor-pinoys'},
 {'link': 'https://newsinfo.inquirer.net/1669546/hrw-disputes-pnps-bloodless-antidrug-efforts-this-month'},
 {'link': 'https://newsinfo.inquirer.net/1669537/masungi-gets-24-7-pnp-patrol-after-abalos-visit'},
 {'link': 'https://newsinfo.inquirer.net/1669545/metro-manila-covid-cases-continue-to-rise-octa'},
 {'link': 'https://newsinfo.inquirer.net/1669543/doj-taking-terror-tag-bid-vs-reds-to-ca'},
 {'link': 'https://newsinfo.inquirer.net/1669555/pogo-workers-now-need-pnp-clearance'},
 {'link': 'https://newsinfo.inquirer.net/1669593/stricter-measures-vs-fake-online-deliveries-urged'}]

In [30]:
%%time

def getArticle(url):
    page = requests.post(url, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    
    title = soup.find_all("h1")[0].text
    
    try:
        author = soup.find(id="art_author").text
    except:
        author = None
        
    
    
    try:
        details = soup.find(id="art_plat").text
    except:
        details = None
    
    for i in soup.find(id="article_content").find_all("p"):
        for tag in i.select("div"):
            tag.decompose()
            
    sentences = []
    
    for i in soup.find(id="article_content").find_all("p"):
        if("RELATED STORY" not in i.text and "Subscribe to INQUIRER PLUS" not in i.text):
            sentences.append(i.text)
        else:
            break
    sentences = " ".join(sentences)

    try:
        date = datetime.datetime.strptime(details.split("/")[1].strip(), '%H:%M %p %B %d, %Y').strftime('%m/%d/%Y')
    except:
        date = None
    
    article = {
        "url":url,
        "title":title,
        "author":author,
        "details":details,
        "date":date,
        "sentences":sentences
    }
    
    return article

sample_article = getArticle('https://newsinfo.inquirer.net/1669041/court-rebuffs-doj-strikes-down-terror-tag-on-cpp-npa')
sample_article

Wall time: 1.41 s


{'url': 'https://newsinfo.inquirer.net/1669041/court-rebuffs-doj-strikes-down-terror-tag-on-cpp-npa',
 'title': 'Court rebuffs DOJ, strikes down terror tag on CPP-NPA',
 'author': 'By: Angel Yabut - @inquirerdotnet\n',
 'details': ' Philippine Daily Inquirer / 05:30 AM September 23, 2022',
 'date': '09/23/2022',
 'sentences': ' MANILA, Philippines — The Manila Regional Trial Court (RTC) has dismissed a 2018 petition by the Department of Justice (DOJ) to declare the Communist Party of the Philippines (CPP) and its armed wing, the New People’s Army (NPA) terrorist organizations, saying the rebel groups were waging a rebellion and not spreading terror. In its Sept. 21 decision, released on Thursday, the Manila RTC Branch 19 ruled that the nine incidents of alleged atrocities by the CPP-NPA cited by the DOJ to back its petition did not meet the requirement for terrorism as defined by the Human Security Act (HSA) of 2007. The incidents presented by the DOJ included five separate killings of

In [6]:
%%time

def getAnnotations(text, lang="en", threshold=0.8):
    
    #ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
    #            "human settlement", "geographic entity", "territorial entity type", "organization"]
    ENTITY_TYPES = ["human", "person"]
    
    url = "http://www.wikifier.org/annotate-article"
    
  
    data = {"userKey":"cutxdoybsvzizpipedvebrhiobbagt",
            "text": text,  
            "lang": lang,
            "pageRankSqThreshold": "%g" % threshold,
            "applyPageRankSqThreshold" : "true",
            "nTopDfValuesToIgnore":"100",
            "nWordsToIgnoreFromList":"100",
            "wikiDataClasses":"true",
            "wikiDataClassIds":"false",
            "support":"true",
            "ranges":"false",
            "minLinkFrequency":"2",
            "includeCosines": "false",
            "maxMentionEntropy":"3"
            }
    results = requests.post(url, data=data, headers=headers)
    
    annotations = list()
    for annotation in json.loads(results.content)["annotations"]:

        # Filter out desired entity classes
        if ('wikiDataClasses' in annotation) and (any([el['enLabel'] in ENTITY_TYPES for el in annotation['wikiDataClasses']])):

            # Specify entity label
            if any([el['enLabel'] in ["human", "person"] for el in annotation['wikiDataClasses']]):
                label = 'Person'
            else:
                label = None
            

            annotations.append({
                'title': annotation['title'], 
                'url':annotation['url'],
                'wikiId': annotation['wikiDataItemId'], 
                'label': label,
                'pageRank' : annotation['pageRank'],
                'characters': [(el['chFrom'], el['chTo']) for el in annotation['support']]})
    
    return annotations

getAnnotations("But is the nation happy?” The remarks of Sen. Francisco Tatad sum up what has generally been mixed reactions to the acquittal of Imelda Marcos.")

Wall time: 1.89 s


[{'title': 'Francisco Tatad',
  'url': 'http://en.wikipedia.org/wiki/Francisco_Tatad',
  'wikiId': 'Q555481',
  'label': 'Person',
  'pageRank': 0.07563996902317344,
  'characters': [(46, 60)]},
 {'title': 'Imelda Marcos',
  'url': 'http://en.wikipedia.org/wiki/Imelda_Marcos',
  'wikiId': 'Q285536',
  'label': 'Person',
  'pageRank': 0.09586283197397719,
  'characters': [(129, 134), (129, 141)]}]

In [7]:
def getTokens(article):

    tokens = []
    for i,value in enumerate(tokenize.sent_tokenize(article['sentences'])):
        annotations = getAnnotations(value, lang="en", threshold=0.8)
        sentence_token = {
            'url': article['url'],
            'token' : value,
            'tokenNo' : i,
            'labels' : [i['label'] for i in annotations],
            'annotations' : annotations    
        }

        tokens.append(sentence_token)
    return tokens

tokens = getTokens(sample_article)
tokens

[{'url': 'https://newsinfo.inquirer.net/1669041/court-rebuffs-doj-strikes-down-terror-tag-on-cpp-npa',
  'token': ' MANILA, Philippines — The Manila Regional Trial Court (RTC) has dismissed a 2018 petition by the Department of Justice (DOJ) to declare the Communist Party of the Philippines (CPP) and its armed wing, the New People’s Army (NPA) terrorist organizations, saying the rebel groups were waging a rebellion and not spreading terror.',
  'tokenNo': 0,
  'labels': [],
  'annotations': []},
 {'url': 'https://newsinfo.inquirer.net/1669041/court-rebuffs-doj-strikes-down-terror-tag-on-cpp-npa',
  'token': 'In its Sept. 21 decision, released on Thursday, the Manila RTC Branch 19 ruled that the nine incidents of alleged atrocities by the CPP-NPA cited by the DOJ to back its petition did not meet the requirement for terrorism as defined by the Human Security Act (HSA) of 2007.',
  'tokenNo': 1,
  'labels': [],
  'annotations': []},
 {'url': 'https://newsinfo.inquirer.net/1669041/court-re

In [34]:
def generateData(start,stop):
    links=[]
    articles=[]
    tokens=[]
    for i in range(start,stop+1):
    
        # generate 9 links
        temp_links = getLinks(i)

        # iterate thru links
        for j in temp_links:

            # generate article from link
            try:
                temp_article = getArticle(j['link'])
            except:
                break
            articles.append(temp_article)

            temp_tokens = getTokens(temp_article)
            tokens = tokens + temp_tokens
        links = links + temp_links

    return links,articles,tokens

In [8]:
tokens_df = pd.DataFrame(tokens)

In [10]:
tokens_df[tokens_df['labels'].str.len()>=2].apply(lambda x: x['annotations'][0]['characters'],axis=1)

11    [(90, 110)]
dtype: object

# Extraction Flow
1. Get Links
2. Iterate Links to get Articles
3. Tokenize Articles to sentences
4. Apply Wikifier to sentences
5. Filter only to sentences with Two person entities identified
6. Apply relation Extraction

In [23]:
%%time


a,b,c = generateData(5,20)

Wall time: 1h 9min 24s


In [27]:
c_df = pd.DataFrame(c)
c_df[c_df['labels'].str.len()==2].shape

(41, 5)

In [60]:
sample_final_df.to_excel("sample_final.xlsx",index=False)

In [61]:
sample_final_df.to_pickle("sample_final.pkl")

In [59]:
sample_final_df

,url,token,tokenNo,labels,annotations
119,https://newsinfo.inquirer.net/1711061/elderly-...,Image from the BuCor website While awaiting th...,1,"[Person, Person]","[{'title': 'Ferdinand Marcos', 'url': 'http://..."
149,https://newsinfo.inquirer.net/1710768/doh-sees...,The Department of Health (DOH) said it was re...,0,"[Person, Person]","[{'title': 'Ferdinand Marcos', 'url': 'http://..."
177,https://newsinfo.inquirer.net/1710775/marcos-r...,PHOTO: President Ferdinand ‘Bongbong’ Marcos ‘...,0,"[Person, Person]","[{'title': 'Bongbong Marcos', 'url': 'http://e..."
223,https://newsinfo.inquirer.net/1710771/marcos-t...,| PHOTO: official pacebook page of President F...,1,"[Person, Person]","[{'title': 'Bongbong Marcos', 'url': 'http://e..."
252,https://newsinfo.inquirer.net/1710429/manila-b...,The agreement will be signed by Foreign Secret...,4,"[Person, Person]","[{'title': 'Enrique Manalo', 'url': 'http://en..."
...,...,...,...,...,...
3305,https://newsinfo.inquirer.net/1679726/dilg-chi...,Southern Police District director Col. Kirby J...,7,"[Person, Person]","[{'title': 'John Kirby (baseball)', 'url': 'ht..."
3421,https://newsinfo.inquirer.net/1679712/doj-chie...,Juanito Jose Diaz Remulla III was arrested by ...,2,"[Person, Person]","[{'title': 'Juanito (footballer, born 1976)', ..."
3471,https://newsinfo.inquirer.net/1679362/hontiver...,Senator Jose “Jinggoy” Estrada is urging the S...,10,"[Person, Person]","[{'title': 'Jinggoy Estrada', 'url': 'http://e..."
3512,https://newsinfo.inquirer.net/1679192/binay-fi...,"During the presidential campaign, President Fe...",27,"[Person, Person]","[{'title': 'Ferdinand Marcos', 'url': 'http://..."
